In [1]:
from functools import wraps
import pandas as pd
import re
import time
import pickle
import ipywidgets as widgets
from IPython.display import display
import csv

import asyncio


In [2]:
fin = pd.read_csv('findata.csv')
fin

,date,month,desc,figure,balance,category
0,2020-05-04,5,"NETONNET AB, VÄXJÖ",-539.00,25556.73,alcohol
1,2020-05-04,5,"NETONNET AB, VÄXJÖ",-200.00,26095.73,alcohol
2,2020-04-30,4,"VAXJO OAS CAFE, VAXJO",-98.00,26295.73,alcohol
3,2020-04-30,4,"PRESSBYRÅN 4025376, VÄXJÖ",-70.00,26393.73,alcohol
4,2020-04-30,4,"kontaktlös LIDL 131/VEXJO OSTERLEDEN, VEXJO",-41.70,26463.73,alcohol
...,...,...,...,...,...,...
225,2020-01-02,1,"WILLYS VÄXJÖ I11, VÄXJÖ",-1200.41,25103.27,NaN
226,2020-01-02,1,"SYSTEMBOLAGET VÄXJÖ, VÄXJÖ",-393.00,26303.68,NaN
227,2020-01-02,1,"WILLYS VÄXJÖ I11, VÄXJÖ",-23.80,26696.68,NaN
228,2020-01-02,1,"JULA AB, VÄXJÖ",-199.00,26720.48,NaN


In [3]:
fincat = {'literature': [], 
          'revolut': ['REVOLUT'], 
          'charity': [],
          'utilities': [], 
          'alcohol': ['SYSTEMBOLAGET'], 
          'holidays': [], 
          'gifts': [], 
          'furniture': [], 
          'tools': [], 
          'electro': [], 
          'phone': [], 
          'rent': [], 
          'car': [], 
          'food': ['WILLYS', 'LIDL', 'PRESSBYRÅN', 'MAX'], 
          'transportation': [], 
          'trips_home': [], 
          'clothes': [], 
          'freetime': [], 
          'exercise': [], 
          'health': [], 
          'savings': [], 
          'misc': [],
          'salary' : []}


In [16]:
category_in = widgets.ToggleButtons(
            options= fincat.keys(),
            description='Category:',
            disabled=False,
            button_style='', 
            tooltips=[],
            )

key_in_text = widgets.Text(
            placeholder='',
            description='New keyword:',
            disabled=False
            )

key_in_button = widgets.ToggleButtons(
                options= ['one', 'two'],
                description='New keyword:',
                disabled=False,
                button_style='',
                )

from ipywidgets import Button, HBox #<----- Add HBox for displaying multiple buttons

def wait_for_change(widget1): #<------ Rename to widget1, and add widget2
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget1.on_click(getvalue, remove=True) #<------ Rename to widget1
        # we need to free up the binding to getvalue to avoid an IvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget1.on_click(getvalue) #<------ Rename to widget1
    return future

button1=Button(description="Next", button_style='success') #<------ Rename to button1

async def categorize(df = fin, d = fincat):
    if not 'category' in df.columns:
        df['category'] = 'NaN'
        
    matches = 0
    keyin_text = '##############################'


    for i, row in df.iterrows():
        
        match = False
        key_iter = 0
        total_len = sum(len(v) for v in iter(d.values()))
        
        key_in_text = widgets.Text(
            placeholder=row['desc'],
            description='New keyword:',
            disabled=False
        )

        key_in_button = widgets.ToggleButtons(
            options= row['desc'].split(),
            description='New keyword:',
            disabled=False,
            button_style='',
        )

        for category, keylist in d.items():
            if match == True:
                match = False
                break

            for key in keylist:
                
                key_iter += 1

                if str(row['desc']).find(key) != -1:
                    matches += 1
                    df.loc[i,'category'] = category
                    print('MATCH!!!')
                    match = True
                    break

                elif (key_iter >= total_len) & (keyin_text == 'break'):
                    break

                elif key_iter >= total_len:
                    
                    display(key_in_text, key_in_button)
                    x = await wait_for_change(button1) #<---- Pass both buttons into the function
                    
                    catin = category_in.value
                    keyin_text = key_in_text.value
                    keyin_button = key_in_button.value
                    
                    if keyin_text == 'break':
                        break
                    elif key_in_text == '':
                        d[catin].append(str(keyin_text))
                    else:
                        d[catin].append(str(keyin_button))
                        df.loc[i,'category'] = catin
                        print('MATCH!!!')
                        match = True
    print(matches, fin, fincat)

asyncio.create_task(categorize(fin, fincat))
HBox([button1, category_in]) #<----Display both buttons in an HBox

Text(value='', description='New keyword:', placeholder=' NETONNET AB, VÄXJÖ')

ToggleButtons(description='New keyword:', options=('NETONNET', 'AB,', 'VÄXJÖ'), value='NETONNET')

MATCH!!!
MATCH!!!


Text(value='', description='New keyword:', placeholder=' VAXJO OAS CAFE, VAXJO')

ToggleButtons(description='New keyword:', options=('VAXJO', 'OAS', 'CAFE,', 'VAXJO'), value='VAXJO')

...

Text(value='', description='New keyword:', placeholder=' Klarna *Itsajten, Kalmar')

ToggleButtons(description='New keyword:', options=('Klarna', '*Itsajten,', 'Kalmar'), value='Klarna')

MATCH!!!
MATCH!!!
MATCH!!!
MATCH!!!
MATCH!!!
MATCH!!!


Text(value='', description='New keyword:', placeholder=' JULA AB, VÄXJÖ')

ToggleButtons(description='New keyword:', options=('JULA', 'AB,', 'VÄXJÖ'), value='JULA')

In [17]:
fincat

{'literature': [],
 'revolut': ['REVOLUT'],
 'charity': ['MEDLEY'],
 'utilities': [],
 'alcohol': ['SYSTEMBOLAGET'],
 'holidays': [],
 'gifts': [],
 'furniture': [],
 'tools': [],
 'electro': ['NETONNET', 'WEBHALLEN.COM,'],
 'phone': [],
 'rent': ['ARABYBOST'],
 'car': [],
 'food': ['WILLYS', 'LIDL', 'PRESSBYRÅN', 'MAX', 'BURGER', 'ICA'],
 'transportation': [],
 'trips_home': [],
 'clothes': [],
 'freetime': ['CAFE,'],
 'exercise': [],
 'health': [],
 'savings': ['8807910'],
 'misc': ['Swish',
  'BUUKS.SE,',
  'Autogiro',
  'LÖN',
  'Elgiganten.se,',
  'MEDIA',
  'CYKELGEAR.SE,',
  'CYKEL',
  'WIKIPEDIA',
  'kontaktlös',
  'CDON',
  'ZALANDO',
  '16127',
  'MMP',
  'INTERSPORT',
  'CLAS',
  'Stanek',
  'BILTEMA',
  'Insättning',
  'EPIDOTEN',
  'BG',
  'ESPRESSO',
  'Månadsavgift',
  'PG',
  'LINNEAMOTTAGNINGEN,',
  'LANDSTINGET',
  'Överföring',
  'DUSTIN',
  'HM.COM,',
  'HOBBYISTERNA',
  'NETFLIX.COM,',
  'CIRCLE',
  'APOTEK',
  'COOP',
  '?HL?NS',
  'LIVETS',
  'CYRANO,',
  'HEMKÖP